# ARROW Python exercises - Activities 9 and 10

1. Read in the CSV files you generated in Activities 7 and 8
2. Use the function provided (vrad_to_lsr()) to correct to the LSR frame of reference.
3. Display the spectrum, identify the peaks and estimate each peak velocity.
4. Save these LSR corrected peak valuse to a file. You can keep a 'running' file going by appending results.

You will need to enter a number of bits of informtion about your observing position, date, and time to do the LSR correction. However, all that data is available in the CSV header so:

OPTIONAL 1: Abstract the data needed for the LSR correction from the header lines. (Otherwise you'll need to input them by hand)

OPTIONAL 2: You can estimate the peak velocities by eye, but there is a matplotlib function called "ginput()" that allows you, on an interactive plot, to mark points on the plot and return an accurate set of coordinates. To get an interactive plot you need to include the following lines at the top of your code, BEFORE using, for example, 'import matplotlib.pyplot as plt':

`

    %matplotlib notebook
    import matplotlib
    matplotlib.use('TkAgg') # provides 'external plots'
`

OPTIONAl 3: Using a ploting module other than matplotlib (or derivative), use a pre-prepared list of CSV files and loop through these in turn. Bokeh would be a suitable plotting module.

NOTE TO OU - makes sense to read in data, do the LSR correction and then measure the peaks. All the information needed is in the header of the CSV file. HOWEVER, this will require a rearrangement of the work flow.

Also, we could do the arm position calculations here as well.


In [1]:
%matplotlib notebook
#import matplotlib
#matplotlib.use('TkAgg') # provides 'external plots'

In [2]:
import pandas as pd
import numpy as np
import re

from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time
from astropy import units as u

from bokeh.plotting import figure, output_notebook, show
from bokeh.models.tools import HoverTool

from IPython.display import display, clear_output

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

output_notebook()  # Needed to get Bokeh plots in the Jupyter notebooks.


Loading BokehJS ...

In [3]:
# Set up the ARROW telescope location as a constant
ARROW = EarthLocation(lat=52.024444*u.deg, \
                   lon=-0.706388*u.deg, \
                   height=114*u.m)


**Function to do LSR correctoon** The following function will be provided - either as a file or possibly as an OU 'package' available via conda.


In [4]:
def vrad_to_lsr(obs_ra, obs_dec, v_rad, location, time):
    """
    Convert a radial velocity to radial velocity in the local 
    standard of rest frame (LSR).

    Parameters
    ----------
    ra : : RA in decimal degrees
    dec : : Dec in decimal degrees
    v_rad : :class:`~astropy.units.Quantity`
        local (from Earth) line-of-sight velocity. km/s
    location : :class: `~astropy.coordinates.EarthLocation`
        location of observation
    time : :class: `astropy.time.Time
        date/time of observation

    Returns
    -------
    v_lsr : :class:`~astropy.units.Quantity`
        Radial velocity in a LSR rest frame.
    
    Based on https://advlabwiki.johnshopkins.edu/images/Vlsr.pdf

    """
    from astropy.coordinates import SkyCoord, EarthLocation
    from astropy.time import Time
    from astropy import units as u
    import numpy as np
    
    
    # We need some basic data
    # LSR correction usualy towards (roughly) Vega
    VEGA_RA=270*u.deg
    VEGA_DEC=30*u.deg
    SUN_VEL = 20.0*u.km/u.s  #Generally accepted speed of sun towards Vega
    
    coordinates = SkyCoord(ra=obs_ra*u.degree, dec=obs_dec*u.degree, frame='icrs')
    
    v_earth_corr = coordinates.radial_velocity_correction(obstime=time, location=location)
    
    ra = coordinates.ra
    dec = coordinates.dec

    v_sun_corr = 0.0*u.km/u.s
    v_sun_corr = SUN_VEL*(np.cos(VEGA_RA)*np.cos(VEGA_DEC)*np.cos(ra)*np.cos(dec) +\
                              np.sin(VEGA_RA)*np.cos(VEGA_DEC)*np.sin(ra)*np.cos(dec) + \
                              np.sin(VEGA_DEC)*np.sin(dec))
    
    v_rad=v_rad*u.km/u.s
    v_lsr = v_rad - v_earth_corr - v_sun_corr

    return v_lsr

**Function to read in the header lines.** Intitially, get the 'header' comments which can, **optionally**, be used later to get information about the observation that will be used to correct to the LSR. Then, use pandas to get the actual data.

We know that usually the first 12 lines are comment header lines, so set that as default.

In [5]:
def get_header(file_name, number_header_lines=12):
    ''' Returns the first number_header_lines as alist '''

    #file_name = input('Enter spectrum file name ')
    #file_name = '080-orig-vel.csv' # For debugging purposes to save typing
    
    with open(file_name) as f:
        header_lines = [f.readline() for x in range(number_header_lines)]
    
    return header_lines

**OPTIONAL - get observational information function ** from the header lines. You could just ask the user for this information using 'input()'

In [22]:
# If you don't do this you'll need to input them manually.
def get_obs_data(header_lines):
    ''' Given a list of ARROW header lines, returns observatio:
            DateTime string
            RA string
            Dec string
        as a list
        '''

    obs_str = ''
    obs_ra=''
    obs_dec=''

    for line in header_lines:
        ss=r"^#startTimeUTC=(.*T.*)Z.*$"
        m=re.search(ss, line.strip())
        if m:
            obs_str=m.groups()[0]
    
        ss=r"^#ra=(\d*\.\d*)"
        m=re.search(ss, line.strip())
        if m:
            obs_ra=float(m.groups()[0])
        ss=r"^#dec=(-*\d*\.\d*)"
        m=re.search(ss, line.strip())
        if m:
            obs_dec=float(m.groups()[0])
    return [obs_str, obs_ra, obs_dec]


**LSR correction** to 'velocity' information' 

In [7]:

#ARROW location:

# Set up date/time in correct format
#obs_dt = Time(obs_str, format='isot', scale='utc')

# Set up x,y (velocity and intensity) values, correcting velocity for LSR.
#velocity=vrad_to_lsr(obs_ra, obs_dec,spectrum_df['velocity'], ARROW,obs_dt)
#intensity=spectrum_df['intensity']


**MAIN LOOP to Plot the spectrum** and **note down** the estimated peaks velocities for each peak at each longitude


In [23]:
# Get list of spectra we'll be using:
vel_files_name = 'vel_files.txt'
v_files=[]
with open(vel_files_name) as vfs:
    v_files=vfs.read().splitlines()

# We'll use a dictionary to store peak velocities. Key = longitude, value list of vels
peak_data = {}
no_header_lines = 12
# THE MAIN LOOP
for file in v_files:
    header = get_header(file, no_header_lines)
    o_data = get_obs_data(header)
    obs_str = o_data[0]
    obs_ra = o_data[1]
    obs_dec = o_data[2]

    # Get main data
    spectrum_df = pd.read_csv(file, header=no_header_lines)
    # Set up date/time in correct format
    obs_dt = Time(obs_str, format='isot', scale='utc')

    # Set up x,y (velocity and intensity) values, correcting velocity for LSR.
    velocity=vrad_to_lsr(obs_ra, obs_dec,spectrum_df['velocity'], ARROW,obs_dt)
    intensity=spectrum_df['intensity']
    
    #Assume longitude is 3 digits in file name.
    # A bit quick and dirty this .....
    long_str=re.search('\d\d\d',file)[0]
    
    p1 = figure(title = "Spectral data from longitude"+long_str, 
          x_axis_label='Velocity (kms^-1)', 
          y_axis_label='Intensity')
    p1.line(velocity,intensity)
    p1.add_tools(HoverTool(mode='vline'))
    show(p1)
    # Wait whilst we input longitude v_obs_max and uncertainty
    print('Enter the peak velocities separated by spaces')
    v_list = list(map(float, input('Enter: peak velocities ').split()))
    peak_data[long_str]=v_list
    # Clear the display before starting again - otherwise we get multiple plots
    clear_output(wait=True)
    

['2014-06-21T12:15:00.566', 7.7317588, -23.8903819]


GlyphRenderer(id='2681', ...)

Enter the peak velocities separated by spaces
Enter: peak velocities 6 7


**OPTIONAL Write/append** the velocity data to a file that can be read in later to plot the Galactic arms.

In [24]:
# Turn the dictionary into a pandas datframe - with peak velocities along rows
# short rows will be 'padded' with NaN
peak_data_df = pd.DataFrame.from_dict(peak_data, orient='index')

# Write to file
peak_data_df.to_csv('cloud_vels_2.csv')